# Rapport TP3 Essaim Particulaire

In [488]:
import numpy as np 
import copy
import numpy.random as rnd
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm


In [489]:
def cout_soudure(x):
    return 1.7781*0.625*x[1]*x[2]**2 + 0.6224*0.625*x[0]*x[2]*x[3] + 3.1661*(0.625*x[0])**2*x[3] + 19.84*(0.625*x[0])**2*x[2]


In [490]:
def g(i,x): #contraintes
    return { 1 : 0.00954*x[2]-0.625*x[1], \
             2 :  0.0193*x[2]- 0.625*x[0], \
             3 : -np.pi * x[2]**2 * x[3] - 4/3 * np.pi * x[2]**3 + 1296000 }[i]


La fonction pénalité permet de calculer le dépassement des contraintes par la particule s'il y a lieu. Elle est simplement la somme des valeurs positives des fonctions contraintes. Cette technique de pénalisation adaptative est recommandée pour les problèmes avec contraintes par les lectures scientifique referees à la fin de ce notebook.

In [491]:
def penalite(x):
    p = [max(g(i,x),0) for i in range(1,4)]

    return np.sum(p)



#### La fonction d'initialisation

In [492]:
def initiation(fobj,bornes,popsize):
   
    d=len(bornes) 
    pop=np.empty([popsize,0]) 
    pop_v= np.empty([popsize,0])
    pop_best_scores = np.empty(shape = popsize) 
    
    for i in range(d):
        column = rnd.uniform(low= bornes[i][0], high = bornes[i][1], size = popsize).reshape(popsize,1)
        pop = np.append(pop,column,axis=1)
        column_v = rnd.uniform(low= -abs(bornes[i][1]-bornes[i][0]), high = abs(bornes[i][1]-bornes[i][0]), size = popsize).reshape(popsize,1)
        pop_v = np.append(pop_v,column_v,axis=1)
    
    pop_best=copy.deepcopy(pop)
    pop_best = [ {'particule' : pop_best[i], 'faisable' : penalite(pop_best[i]) == 0 } for i in range(len(pop_best))] # convertir les elements en une liste de dictionnaires qui nous permet de savoir si la particule respeste les contraintes ou pas

    
    for i in range(popsize):
        pop_best_scores[i] = fobj(pop[i]) + penalite(pop[i])
            
    global_best=pop[np.argmin(pop_best_scores)]


    

    return d,pop, pop_best, global_best, pop_v, pop_best_scores



#### Vérification des bornes

In [493]:
def print_results(hist):
    
    scores = hist[:,1]
    best_score = min(scores)
    index = np.where(scores == best_score)
    #print(f"index : {index}")
    print(f"meilleurs dimensions : \n z1 = {0.625*hist[index[0][0],4]} \n z2 = {0.625*hist[index[0][0],5]} \n x3 = {hist[index[0][0],6]} \n x4 = {hist[index[0][0],7]}\n\n faisable = {'oui' if hist[index[0][0],2] == 1 else 'non'} \n")
    print(f"meilleur score : {best_score}\n")

   

In [494]:
def preparation_table(hist):
    hist[:,3] = [0.625 * a for a in hist[:,3]]
    hist[:,4] = [0.625 * a for a in hist[:,4]]

    table = pd.DataFrame(data=hist[:,1:], columns=["MeilleurScore","Faisabilité","NbrItrs", "z1","z2","x3","x4"], index=hist[:,0]) 
    
    return table





In [495]:
def print_table(hist):
      print("historique detaille :\n")
      return preparation_table(hist)



In [496]:
def print_analyse(hist):
    print("Analyse des donnees generees :\n")
    return preparation_table(hist).describe()


#### "faisable" : respecte les contraintes.

En comparant deux particules, une particule est meilleure si son score est inferieur et que les deux particules sont faisables ou que sa pénalité est inférieure et que les deux particules ne sont pas faisables. 

Dans le cas où un est faisable et l'autre non, on préfère toujours celle qui est faisable sans considérer le score.

Dans ma comparaison de la nouvelle position de la particule générée avec sa meilleure position, au lieu de condenser le code en vérifiant plusieurs conditions, j'ai séparé les conditions pour simplifier la lecture du code.

Le nombre d'itérations fixe par l'utilisateur n'est pas garanti d'être le nombre maximal d'itérations étant donné qu'on ne pas avoir un optimum global faisable à la fin de ces itérations. Dans ce cas, on continue à itérer jusqu'à ce qu'on tombe sur un optimum global faisable.



In [497]:
import numpy as np
a = np.empty([3,0])
b = np.array([1,2,3]).reshape(3,1)
a = np.append(a, b, axis=1)
a

array([[1.],
       [2.],
       [3.]])

In [498]:
def particleswarm(fobj,exec, iter, bornes,popsize,c1,c2,wmin,wmax,vc,tol):
 
    historique = np.empty([0,8])
    for k in tqdm(range(exec)):
        ############initialisation
        d,pop, pop_best, global_best, pop_v, pop_best_scores = initiation(fobj,bornes,popsize)
                
        j=0
        while (j < iter or abs(fobj(old_global_best)-fobj(global_best))>tol) or penalite(global_best) > 0 :
                
                w = wmax - (wmax-wmin) * j / iter #propose par Shi https://www.matec-conferences.org/articles/matecconf/pdf/2016/26/matecconf_mmme2016_02019.pdf
                
                vmax = np.exp((1-j)/iter)*vc # recherches sur le VPSO https://www.matec-conferences.org/articles/matecconf/pdf/2016/26/matecconf_mmme2016_02019.pdf
                
                j+=1
            
                for i in range(popsize):
                    r1,r2=rnd.uniform(0,1,2) 
                    
                    pop_v[i] = pop_v[i]*w +  (c1*r1*(pop_best[i]['particule']-pop[i])) + (c2*r2*(global_best-pop[i]))
                    
                    #verification des bornes de la velocite
                    for l in range(len(pop_v[i])):
                        if pop_v[i][l] > vmax : 
                             pop_v[i][l] = vmax
                        
                        elif pop_v[i][l] < -vmax : 
                             pop_v[i][l] = -vmax
                    
                    
                    pop[i] += pop_v[i] 
                    
                    #verification des bornes de la position
                    for l in range(len(bornes)):
                        
                        if pop[i][l] < bornes[l][0]: 
                             pop[i][l] = bornes[l][0]
                        
                        elif pop[i][l] > bornes[l][1]: 
                             pop[i][l] = bornes[l][1]
                    

                    penal = penalite(pop[i])
                    particle_fitness= fobj(pop[i]) + penal # penalite = 0 si la particule respecte les contraites
                
                    
                    if  penal == 0 and pop_best[i]['faisable'] :
                            
                            if particle_fitness < pop_best_scores[i] :
                                pop_best[i]['particule']=pop[i,:] #checking if new best
                                pop_best_scores[i]=particle_fitness
                                
                    elif penal >0 and not pop_best[i]['faisable'] :
                               
                               if penal < penalite(pop_best[i]['particule']):
                                   pop_best[i]['particule']=pop[i,:] #checking if new best
                                   pop_best_scores[i]=particle_fitness
   
                    elif penal == 0 and not pop_best[i]['faisable'] : 
                            
                                pop_best[i]['particule']=pop[i,:] #checking if new best
                                pop_best_scores[i]=particle_fitness
                                


                    penal_g = penalite(global_best)
                    global_best_fitness = fobj(global_best) + penal_g           
                    
                    if (penal >0 and penal_g > 0  and penal < penal_g) or (penal == 0 and penal_g == 0 and particle_fitness < global_best_fitness) or (penal == 0 and penal_g >0 ): 

                                    old_global_best=global_best[:]
                                    global_best=copy.deepcopy(pop_best[i]['particule']) 
                                    #print('current function value: ',fobj(global_best))
 
                    
    
    #print('Optimum at: ',global_best,'\n','Function at optimum: ',fobj(global_best)) 

        historique = np.append(historique,[np.append([k,fobj(global_best),penalite(global_best)==0,j],[global_best])],axis=0)
        
        print(f'#global best monte-carlo iter#{k} is : {fobj(global_best)}')
    
    return historique


In [499]:
fobj=cout_soudure

#https://www.matec-conferences.org/articles/matecconf/pdf/2016/26/matecconf_mmme2016_02019.pdf

bornes = [[1,99],[1,99],[10,200],[10,240]]    
popsize=20 

c1=2.8 # Clerc et autres chercheurs 
c2=2.05 #ont abouti par experimentation a ce resultat
vc = 1.5
wmin = 0.4
wmax = 0.9
tol=0.001
exec =100
iter = 2000

In [500]:
while True:
    iter = input("Entrer le nombre d'iterations par execution (mieux pour 2000): ")
    epsilon = input("Entrer epsilon ( progres minimal / mieux pour 0.001 ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Veuillez reesayer!")

hist = particleswarm(fobj,exec,int(iter),bornes,popsize,c1,c2,wmin,wmax,vc,float(epsilon))    


  1%|          | 1/100 [00:06<11:09,  6.76s/it]

#global best monte-carlo iter#0 is : 6990.33197291997


  2%|▏         | 2/100 [00:12<10:28,  6.42s/it]

#global best monte-carlo iter#1 is : 6648.702972394803


  3%|▎         | 3/100 [00:19<10:09,  6.28s/it]

...

#global best monte-carlo iter#97 is : 7244.769548690746


 99%|█████████▉| 99/100 [09:56<00:05,  5.79s/it]

#global best monte-carlo iter#98 is : 7114.055504374412


100%|██████████| 100/100 [10:02<00:00,  6.03s/it]

#global best monte-carlo iter#99 is : 6642.599991092781


In [501]:
print_results(hist)



meilleurs dimensions : 
 z1 = 0.727590929353616 
 z2 = 0.625 
 x3 = 37.69901188360704 
 x4 = 240.0

 faisable = oui 

meilleur score : 6474.936062020174



In [502]:
print_table(hist)



historique detaille :



,MeilleurScore,Faisabilité,NbrItrs,z1,z2,x3,x4
0.0,6990.331973,1.0,1875.0,0.990454,1.0,51.318844,88.214414
1.0,6648.702972,1.0,1875.0,0.811579,1.0,42.050720,177.228895
2.0,6488.291553,1.0,1875.0,0.733988,1.0,38.030478,234.520297
3.0,6910.473509,1.0,1875.0,0.945769,1.0,49.003554,106.452756
4.0,6727.794789,1.0,1875.0,0.850756,1.0,44.080624,153.530563
...,...,...,...,...,...,...,...
95.0,6851.678511,1.0,1875.0,0.914292,1.0,47.372645,120.659478
96.0,6795.125335,1.0,1875.0,0.884888,1.0,45.849139,135.110172
97.0,7244.769549,1.0,1875.0,1.162193,1.0,60.217257,33.476508
98.0,7114.055504,1.0,1875.0,1.066196,1.0,55.243296,61.517144


In [503]:
print_analyse(hist)


Analyse des donnees generees :



,MeilleurScore,Faisabilité,NbrItrs,z1,z2,x3,x4
count,100.000000,100.0,100.000,100.000000,100.0,100.000000,100.000000
mean,6819.974734,1.0,1171.875,0.563943,1.0,46.751691,132.617102
std,176.636047,0.0,0.000,0.060587,0.0,5.022703,46.395230
min,6474.936062,1.0,1171.875,0.454744,1.0,37.699012,17.631037
25%,6715.249464,1.0,1171.875,0.527804,1.0,43.755746,106.794778
50%,6802.280911,1.0,1171.875,0.555355,1.0,46.039791,133.234058
75%,6909.043365,1.0,1171.875,0.590614,1.0,48.962625,157.135875
max,7311.739634,1.0,1171.875,0.765997,1.0,63.502161,240.000000


### Conclusion : 

Cette métaheuristique donne de meilleurs résultats par rapport à celles utilisées dans le TP1, et s'approche d'ailleurs de l'optimum expérimentale trouvée pour ce problème qui est dans les environs de 5000.

### REFERENCES :
 1.https://www.cs.cinvestav.mx/~constraint/papers/eisci.pdf
 3.https://www.sciencedirect.com/science/article/pii/S0957417419305925
 4.http://ipedr.com/vol6/29-A10025.pdf
 https://www.matec-conferences.org/articles/matecconf/pdf/2016/26/matecconf_mmme2016_02019.pdf
 https://www.researchgate.net/figure/Pseudocode-of-standard-particle-swarm-optimization_fig1_323281605